Let's starts with the imports

In [1]:
import pandas as pd
from pathlib import Path

import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")


Add a function to load all the BST dataset files.

In [2]:
def load_all_bst(data_dir: Path,
                 state_codes: list[str] = ["NY", "TEX", "GA", "IL"],
                 years: list[int]     = [22, 23, 24]) -> pd.DataFrame:
    """
    Load and concatenate all BST On-Time CSVs from the BST/ subfolder.
    """
    bst_dir = data_dir / "BST"
    frames = []
    for state in state_codes:
        for yr in years:
            csv_path =bst_dir / f"T_ONTIME_REPORTING_{state}_{yr}.csv"
            try:
                logging.info(f"Reading BST file: {csv_path.name}")
                df = pd.read_csv(csv_path, encoding="latin-1")
                frames.append(df)
            except FileNotFoundError:
                logging.warning(f"File not found, skipping: {csv_path.name}")
    if not frames:
        logging.error("No BST files loaded. Check the BST/ folder.")
        return pd.DataFrame()
    combined = pd.concat(frames, ignore_index=True)
    logging.info(f"Combined BST records: {combined.shape[0]:,}")
    return combined


Add a function to load all the NOAA dataset files.

In [3]:
def load_all_noaa(data_dir: Path, pattern: str = "*.csv") -> pd.DataFrame:
    noaa_dir = data_dir / "NOAA"
    files = sorted(noaa_dir.glob(pattern))
    if not files:
        logging.error(f"No NOAA CSVs found in {noaa_dir}.")
        return pd.DataFrame()
    
    frames = []
    for f in files:
        logging.info(f"Reading NOAA file: {f.name}")
        df = pd.read_csv(f, encoding="latin-1", low_memory=False)
        frames.append(df)
    combined_noaa = pd.concat(frames, ignore_index=True)
    logging.info(f"Combined NOAA: {combined_noaa.shape[0]:,} rows from {len(files)} files")
    return combined_noaa


Add clean and filter function to first Lowercase & strip column names then Reconstruct `fl_date` from YEAR/MONTH/DAY_OF_MONTH and Parse departure times (CRS_DEP_TIME & DEP_TIME) Filter out cancelled/diverted flights , Drop rows missing WEATHER_DELAY then Restrict to rows where origin or dest is in target_airport_ids

In [4]:

def clean_and_filter(raw_bst: pd.DataFrame,
                     raw_noaa: pd.DataFrame,
                     target_airport_ids: list[int]) -> pd.DataFrame:

    # lowercase & strip
    raw_bst.columns  = raw_bst.columns.str.strip().str.lower()
    raw_noaa.columns = raw_noaa.columns.str.strip().str.lower()
    
    # Reconstruct fl_date in BST data
    if all(col in raw_bst.columns for col in ["year", "month", "day_of_month"]):
        raw_bst["fl_date"] = pd.to_datetime({
            "year":  raw_bst["year"].astype(int),
            "month": raw_bst["month"].astype(int),
            "day":   raw_bst["day_of_month"].astype(int)
        })
    else:
        raise KeyError("BST columns missing: YEAR, MONTH, DAY_OF_MONTH")
    
    # Parse CRS_DEP_TIME & DEP_TIME into timestamps (format HHMM)
    for col in ("crs_dep_time", "dep_time"):
        if col in raw_bst.columns:
            raw_bst[col] = (
                raw_bst[col]
                .fillna(0)
                .astype(int)
                .astype(str)
                .str.zfill(4)  # e.g. '   5' -> '0005', '  85' -> '0085'
            )
            raw_bst[col] = pd.to_datetime(
                raw_bst["fl_date"].dt.strftime("%Y-%m-%d") + " " + raw_bst[col],
                format="%Y-%m-%d %H%M",
                errors="coerce"
            )
    
    # Filter out cancelled/diverted in BST
    for flag in ("cancelled", "diverted"):
        if flag in raw_bst.columns:
            raw_bst = raw_bst[raw_bst[flag] == 0]
    
    # Drop any rows missing WEATHER_DELAY
    if "weather_delay" in raw_bst.columns:
        raw_bst = raw_bst.dropna(subset=["weather_delay"])
    else:
        raise KeyError("Column 'WEATHER_DELAY' not found in BST data")
    
   
    # Filter to tatget airport ids in origin or dest
    if not {"origin_airport_id", "dest_airport_id"}.issubset(raw_bst.columns):
        raise KeyError("BST must have both ORIGIN_AIRPORT_ID and DEST_AIRPORT_ID")
    
    mask = (
        raw_bst["origin_airport_id"].isin(target_airport_ids) |
        raw_bst["dest_airport_id"].isin(target_airport_ids)
    )
    filtered = raw_bst[mask].copy()
    logging.info(f"Filtered BST rows to targets: {filtered.shape[0]:,}")
    
    return filtered


In [5]:
    # Define the base directory
    base_dir = Path("/work/Data/")
    
    # Load BST & NOAA
    bst_df  = load_all_bst (base_dir)
    noaa_df = load_all_noaa(base_dir)
    
    # Define our four hub airport IDs (ATL, JFK, ORD, DFW)
    target_airport_ids = [10397, 12478, 13930, 11298]
    
    bst_df.head(10)

2025-06-19 05:45:48,438 INFO: Reading BST file: T_ONTIME_REPORTING_NY_22.csv
2025-06-19 05:45:51,540 INFO: Reading BST file: T_ONTIME_REPORTING_NY_23.csv
2025-06-19 05:45:53,550 INFO: Reading BST file: T_ONTIME_REPORTING_NY_24.csv
2025-06-19 05:45:55,242 INFO: Reading BST file: T_ONTIME_REPORTING_TEX_22.csv
2025-06-19 05:45:57,319 INFO: Reading BST file: T_ONTIME_REPORTING_TEX_23.csv
2025-06-19 05:45:59,136 INFO: Reading BST file: T_ONTIME_REPORTING_TEX_24.csv
2025-06-19 05:46:01,411 INFO: Reading BST file: T_ONTIME_REPORTING_GA_22.csv
2025-06-19 05:46:03,287 INFO: Reading BST file: T_ONTIME_REPORTING_GA_23.csv
2025-06-19 05:46:04,514 INFO: Reading BST file: T_ONTIME_REPORTING_GA_24.csv
2025-06-19 05:46:05,726 INFO: Reading BST file: T_ONTIME_REPORTING_IL_22.csv
2025-06-19 05:46:06,560 INFO: Reading BST file: T_ONTIME_REPORTING_IL_23.csv
2025-06-19 05:46:07,573 INFO: Reading BST file: T_ONTIME_REPORTING_IL_24.csv
2025-06-19 05:46:08,800 INFO: Combined BST records: 801,435
2025-06-19 05

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2022,1,1,6,10140,"Albuquerque, NM",NM,New Mexico,12478,JFK,...,0.0,NaN,0.0,233.0,226.0,18.0,0.0,0.0,0.0,50.0
1,2022,1,1,6,10257,"Albany, NY",NY,New York,10397,ATL,...,0.0,NaN,0.0,180.0,157.0,NaN,NaN,NaN,NaN,NaN
2,2022,1,1,6,10257,"Albany, NY",NY,New York,10397,ATL,...,0.0,NaN,0.0,161.0,148.0,NaN,NaN,NaN,NaN,NaN
3,2022,1,1,6,10257,"Albany, NY",NY,New York,10821,BWI,...,0.0,NaN,0.0,95.0,72.0,NaN,NaN,NaN,NaN,NaN
4,2022,1,1,6,10257,"Albany, NY",NY,New York,10821,BWI,...,0.0,NaN,0.0,85.0,73.0,NaN,NaN,NaN,NaN,NaN
5,2022,1,1,6,10257,"Albany, NY",NY,New York,10821,BWI,...,0.0,NaN,0.0,85.0,79.0,NaN,NaN,NaN,NaN,NaN
6,2022,1,1,6,10257,"Albany, NY",NY,New York,11057,CLT,...,0.0,NaN,0.0,150.0,121.0,NaN,NaN,NaN,NaN,NaN
7,2022,1,1,6,10257,"Albany, NY",NY,New York,11057,CLT,...,0.0,NaN,0.0,146.0,135.0,NaN,NaN,NaN,NaN,NaN
8,2022,1,1,6,10257,"Albany, NY",NY,New York,11057,CLT,...,0.0,NaN,0.0,131.0,131.0,NaN,NaN,NaN,NaN,NaN
9,2022,1,1,6,10257,"Albany, NY",NY,New York,11278,DCA,...,0.0,NaN,0.0,85.0,92.0,NaN,NaN,NaN,NaN,NaN


In [6]:
noaa_df.head(10)

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,AX5,KA3,KA4,AW5,AT7,AU5,AW6,AW7,GA4,AT8
0,72219013874,2022-01-01T00:00:00,4,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72219013874,2022-01-01T00:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72219013874,2022-01-01T01:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72219013874,2022-01-01T02:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72219013874,2022-01-01T03:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,72219013874,2022-01-01T04:37:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-16,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,72219013874,2022-01-01T04:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,72219013874,2022-01-01T04:59:00,6,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,SOD,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,72219013874,2022-01-01T04:59:00,6,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,SOM,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,72219013874,2022-01-01T05:10:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-16,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Extract a unique list of (station, name) pairs from NOAA to identify airports
station_name_df = noaa_df[['STATION', 'NAME']].drop_duplicates().copy()

# Normalize station names for easier matching
station_name_df['NAME'] = station_name_df['NAME'].str.upper().str.strip()

# Create a rough mapping manually based on known major hubs (we'll hardcode the known matches)
known_mapping = {
    'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US': 10397,  # ATL
    'JFK INTERNATIONAL AIRPORT, NY US': 12478,             # JFK
    'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US': 13930,              # ORD
    'DAL FTW WSCMO AIRPORT, TX US': 11298           # DFW
}

# Assign airport_id using the known mapping
station_name_df['airport_id'] = station_name_df['NAME'].map(known_mapping)

# Display the dataframe to the user using an alternative method
print("Inferred Station ↔ Airport ID Mapping")
print(station_name_df.head(10))

# Ensure station column is string in both DataFrames
noaa_df['STATION'] = noaa_df['STATION'].astype(str)
station_name_df['STATION'] = station_name_df['STATION'].astype(str)

noaa_df = pd.merge(noaa_df, station_name_df[['STATION', 'airport_id']], on='STATION', how='left')

# Check how many were successfully mapped
mapped_count = noaa_df['airport_id'].notna().sum()
total_count = len(noaa_df)

f"Mapped {mapped_count:,} out of {total_count:,} NOAA records to airport_id."


Inferred Station ↔ Airport ID Mapping
           STATION                                               NAME  \
0      72219013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
12330  72259003927                       DAL FTW WSCMO AIRPORT, TX US   
25688  74486094789                   JFK INTERNATIONAL AIRPORT, NY US   
39032  72530094846         CHICAGO OHARE INTERNATIONAL AIRPORT, IL US   

       airport_id  
0           10397  
12330       11298  
25688       12478  
39032       13930  


'Mapped 158,617 out of 158,617 NOAA records to airport_id.'

In [8]:
# Standardize all column names to lowercase
noaa_df.columns = noaa_df.columns.str.lower()
station_name_df.columns = station_name_df.columns.str.lower()

# Ensure 'station' column is string in both DataFrames
noaa_df['station'] = noaa_df['station'].astype(str)
station_name_df['station'] = station_name_df['station'].astype(str)

noaa_df = noaa_df.drop(columns=['airport_id'], errors='ignore')

# Merge airport_id into NOAA
noaa_df = pd.merge(
    noaa_df,
    station_name_df[['station', 'airport_id']],
    on='station',
    how='left'
)

print(bst_df.columns.tolist())
print(noaa_df.columns.tolist())


['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'DEP_TIME', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY_NEW', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
['station', 'date', 'source', 'latitude', 'longitude', 'elevation', 'name', 'report_type', 'call_sign', 'quality_control', 'wnd', 'cig', 'vis', 'tmp', 'dew', 'slp', 'aa1', 'aa2', 'aa3', 'ab1', 'ad1', 'ae1', 'ah1', 'ah2', 'ah3', 'ah4', 'ah5', 'ah6', 'ai1', 'ai2', 'ai3', 'ai4', 'ai5', 'ai6', 'aj1', 'ak1', 'al1', 'am1', 'an1', 'at1', 'at2', 'at3', 'at4', 'at5', 'au1', 'au2', 'au3', 'aw1', 'aw2', 'aw3', 'aw4', 'ax1', 'ax2', 'ax3', 'ed1', 'ga1', 'ga2', 'ga3', 'gd1', 'gd2', 'gd3', 'gd4', 'ge1', 'gf1', 'ka1', 'ka2', 'kb1', 'kb2', 'kb3', 'kc1', 'kc

In [9]:
# Ensure date columns are in datetime format
noaa_df['date'] = pd.to_datetime(noaa_df['date'], errors='coerce')
noaa_df['timestamp_hour'] = noaa_df['date'].dt.floor('H')

noaa_df.head(10)


,station,date,source,latitude,longitude,elevation,name,report_type,call_sign,quality_control,...,ka4,aw5,at7,au5,aw6,aw7,ga4,at8,airport_id,timestamp_hour
0,72219013874,2022-01-01 00:00:00,4,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 00:00:00
1,72219013874,2022-01-01 00:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 00:00:00
2,72219013874,2022-01-01 01:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 01:00:00
3,72219013874,2022-01-01 02:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 02:00:00
4,72219013874,2022-01-01 03:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 03:00:00
5,72219013874,2022-01-01 04:37:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-16,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 04:00:00
6,72219013874,2022-01-01 04:52:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-15,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 04:00:00
7,72219013874,2022-01-01 04:59:00,6,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,SOD,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 04:00:00
8,72219013874,2022-01-01 04:59:00,6,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,SOM,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 04:00:00
9,72219013874,2022-01-01 05:10:00,7,33.62972,-84.44224,308.2,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,FM-16,KATL,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 05:00:00


In [10]:
# Prepare date columns with required names
date_parts = bst_df[['YEAR', 'MONTH', 'DAY_OF_MONTH']].rename(
    columns={'YEAR': 'year', 'MONTH': 'month', 'DAY_OF_MONTH': 'day'}
)

# Now safely create FL_DATE
bst_df['FL_DATE'] = pd.to_datetime(date_parts, errors='coerce')

def extract_dep_hour(date_series, time_series):
    padded = time_series.fillna(0).astype(int).astype(str).str.zfill(4)
    hours = padded.str[:2].astype(int)
    return date_series + pd.to_timedelta(hours, unit='h')

bst_df['dep_hour'] = extract_dep_hour(bst_df['FL_DATE'], bst_df['CRS_DEP_TIME'])


In [11]:
# Filter both datasets to ATL
bst_atl = bst_df[bst_df['ORIGIN_AIRPORT_ID'] == 10397]
noaa_atl = noaa_df[noaa_df['airport_id'] == 10397]

# Merge only ATL rows
merged_atl = pd.merge(
    bst_atl,
    noaa_atl,
    how='left',
    left_on=['ORIGIN_AIRPORT_ID', 'dep_hour'],
    right_on=['airport_id', 'timestamp_hour']
)
merged_atl

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,...,ka4,aw5,at7,au5,aw6,aw7,ga4,at8,airport_id,timestamp_hour
0,2022,1,1,6,10397,"Atlanta, GA",GA,Georgia,10257,ALB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 10:00:00
1,2022,1,1,6,10397,"Atlanta, GA",GA,Georgia,10257,ALB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 21:00:00
2,2022,1,1,6,10397,"Atlanta, GA",GA,Georgia,10792,BUF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 10:00:00
3,2022,1,1,6,10397,"Atlanta, GA",GA,Georgia,10792,BUF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 21:00:00
4,2022,1,1,6,10397,"Atlanta, GA",GA,Georgia,12197,HPN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2022-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136554,2024,1,31,3,10397,"Atlanta, GA",GA,Georgia,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2024-01-31 18:00:00
136555,2024,1,31,3,10397,"Atlanta, GA",GA,Georgia,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2024-01-31 18:00:00
136556,2024,1,31,3,10397,"Atlanta, GA",GA,Georgia,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2024-01-31 18:00:00
136557,2024,1,31,3,10397,"Atlanta, GA",GA,Georgia,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397,2024-01-31 21:00:00


In [12]:
results = []

for airport_id in [10397, 12478, 13930, 11298]:  # ATL, JFK, ORD, DFW
    bst_chunk = bst_df[bst_df['ORIGIN_AIRPORT_ID'] == airport_id]
    noaa_chunk = noaa_df[noaa_df['airport_id'] == airport_id]

    merged = pd.merge(
        bst_chunk,
        noaa_chunk,
        how='left',
        left_on=['ORIGIN_AIRPORT_ID', 'dep_hour'],
        right_on=['airport_id', 'timestamp_hour']
    )
    results.append(merged)

merged_df = pd.concat(results, ignore_index=True)


In [13]:
merged_df.shape

(430192, 142)

In [14]:
merged_df.to_csv('merged_data.csv', index=False)

In [15]:
#This code consumes a lot of memory and rigger a Deepnote kernel crash 
# Ensure timestamp columns are in datetime format
#noaa_df['timestamp_hour'] = pd.to_datetime(noaa_df['timestamp_hour'], errors='coerce')
#bst_df['dep_hour'] = pd.to_datetime(bst_df['dep_hour'], errors='coerce')

# Perform the merge
#merged_df = pd.merge(
#    bst_df,
#    noaa_df,
#    how='left',
#    left_on=['ORIGIN_AIRPORT_ID', 'dep_hour'],
#    right_on=['airport_id', 'timestamp_hour']
#)
#print(f"Merged rows: {len(merged_df)}")


In [10]:
!pip install xgboost
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 1.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, roc_curve
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import uuid

# Define dtypes for relevant columns to avoid mixed-type warnings
dtypes = {
    'ORIGIN_AIRPORT_ID': 'int32', 'DEST_AIRPORT_ID': 'int32', 'DAY_OF_WEEK': 'int8',
    'DEP_TIME': 'object', 'CRS_DEP_TIME': 'object', 'WEATHER_DELAY': 'float32',
    'TAXI_OUT': 'float32', 'TAXI_IN': 'float32', 'latitude': 'float32', 'longitude': 'float32',
    'wnd': 'object', 'vis': 'object', 'tmp': 'object', 'dew': 'object', 'aa1': 'object'
}

# Load the entire dataset with specified column types and relevant columns
relevant_cols = ['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'DAY_OF_WEEK', 'DEP_TIME', 'CRS_DEP_TIME',
                 'WEATHER_DELAY', 'TAXI_OUT', 'TAXI_IN', 'latitude', 'longitude', 'wnd', 'vis', 'tmp', 'dew', 'aa1']
df = pd.read_csv('/work/merged_data.csv', usecols=relevant_cols, dtype=dtypes)

# Parse weather features with clipping to prevent overflow
def estimate_humidity(temp, dewp):
    temp, dewp = np.clip(temp, -50, 50), np.clip(dewp, -50, 50)  # Clip to reasonable range
    return np.clip(100 * (np.exp((17.625 * dewp) / (243.04 + dewp)) / np.exp((17.625 * temp) / (243.04 + temp))), 0, 100)

def parse_wind(wnd_str):
    return float(wnd_str.split(',')[1].strip()) if isinstance(wnd_str, str) and len(wnd_str.split(',')) >= 2 else np.nan

def parse_visibility(vis_str):
    return 10000.0 if isinstance(vis_str, str) and vis_str.split(',')[0].strip() == '9999' else float(vis_str.split(',')[0].strip()) if isinstance(vis_str, str) else np.nan

def parse_temperature(tmp_str):
    return float(tmp_str.split(',')[0].replace('+', '')) / 10.0 if isinstance(tmp_str, str) else np.nan

def parse_dewpoint(dew_str):
    return float(dew_str.split(',')[0].replace('+', '')) / 10.0 if isinstance(dew_str, str) else np.nan

def parse_precipitation(aa1_str):
    return float(aa1_str.split(',')[1].strip()) if isinstance(aa1_str, str) and len(aa1_str.split(',')) > 1 else 0.0

# Apply parsing with float32 to avoid overflow
df['WindSpeed'] = df['wnd'].apply(parse_wind).astype('float32').clip(0, 100)
df['vis'] = df['vis'].apply(parse_visibility).astype('float32').clip(0, 10000)
df['Temperature'] = df['tmp'].apply(parse_temperature).astype('float32').clip(-50, 50)
df['DewPoint'] = df['dew'].apply(parse_dewpoint).astype('float32').clip(-50, 50)
df['Precipitation'] = df['aa1'].apply(parse_precipitation).astype('float32').clip(0, 100)
df['Humidity'] = np.where((df['Temperature'].notna()) & (df['DewPoint'].notna()),
                          estimate_humidity(df['Temperature'], df['DewPoint']), 50.0).astype('float32').clip(0, 100)

# Feature engineering
dep_hour = pd.to_datetime(df['DEP_TIME'], errors='coerce').dt.hour.fillna(
    pd.to_datetime(df['CRS_DEP_TIME'], errors='coerce').dt.hour).fillna(12).astype('int8')
df['HourSin'] = np.sin(2 * np.pi * dep_hour / 24).astype('float32')
df['HourCos'] = np.cos(2 * np.pi * dep_hour / 24).astype('float32')
df['OriginDest'] = df['ORIGIN_AIRPORT_ID'].astype(str) + '-' + df['DEST_AIRPORT_ID'].astype(str)
df['Delayed'] = (df['WEATHER_DELAY'] > 60).astype('int8')

# Select features
columns_to_keep = ['WindSpeed', 'vis', 'Temperature', 'DewPoint', 'Precipitation', 'Humidity', 'TAXI_OUT', 'TAXI_IN',
                   'latitude', 'longitude', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'OriginDest', 'Delayed']
df = df[columns_to_keep].copy()

# Handle missing values
numeric_cols = ['WindSpeed', 'vis', 'Temperature', 'DewPoint', 'Precipitation', 'Humidity', 'TAXI_OUT', 'TAXI_IN', 'latitude', 'longitude']
categorical_cols = ['DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median()).astype('float32')
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0]).astype('int16')

# Features and target
features = ['OriginDest', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID'] + numeric_cols
X = df[features]
y = df['Delayed']

# Preprocessing with sparse one-hot encoding
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, ['OriginDest', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT_ID'])
], sparse_threshold=0.3)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Apply SMOTE with conservative oversampling
smote = SMOTE(sampling_strategy=0.3, random_state=42, k_neighbors=3)
X_train_res, y_train_res = smote.fit_resample(preprocessor.fit_transform(X_train).toarray(), y_train)  # Convert sparse to dense for SMOTE
X_test_transformed = preprocessor.transform(X_test).toarray()

# Define models with class weights
models = {
    'Logistic Regression': (LogisticRegression(max_iter=500, class_weight='balanced', random_state=42), {'classifier__C': [0.1, 1.0]}),
    'Random Forest': (RandomForestClassifier(class_weight='balanced', random_state=42), {'classifier__max_depth': [10], 'classifier__n_estimators': [50]}),
    'XGBoost': (xgb.XGBClassifier(scale_pos_weight=(y_train==0).sum()/(y_train==1).sum(), random_state=42), {'classifier__max_depth': [6], 'classifier__learning_rate': [0.1], 'classifier__n_estimators': [50]})
}

# Store results
model_results = {}

# Train and visualize
for name, (model, param_grid) in models.items():
    try:
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='roc_auc', n_jobs=1, error_score='raise')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        y_proba = best_model.predict_proba(X_test)[:, 1]

        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        model_results[name] = {'Accuracy': accuracy, 'ROC-AUC': roc_auc, 'Precision': precision, 'Recall': recall}

        print(f"\n{name} Results (Best Params: {grid_search.best_params_}):")
        for metric, value in model_results[name].items():
            print(f"{metric}: {value:.4f}")

        # Visualization 1: Correlation Matrix
        plt.figure(figsize=(8, 6))
        corr = df[numeric_cols].corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
        plt.title(f'Correlation Matrix - {name}')
        plt.tight_layout()
        plt.savefig(f'corr_matrix_{name.lower().replace(" ", "_")}_{uuid.uuid4()}.png')
        plt.close()

        # Visualization 2: ROC-AUC Curve
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        plt.figure(figsize=(6, 6))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC-AUC Curve - {name}')
        plt.legend(loc='best')
        plt.tight_layout()
        plt.savefig(f'roc_auc_{name.lower().replace(" ", "_")}_{uuid.uuid4()}.png')
        plt.close()

        # Visualization 3: Boxplots for Numeric Features
        plt.figure(figsize=(10, 6))
        df[numeric_cols].boxplot()
        plt.xticks(rotation=45)
        plt.title(f'Boxplots of Numeric Features - {name}')
        plt.tight_layout()
        plt.savefig(f'boxplot_{name.lower().replace(" ", "_")}_{uuid.uuid4()}.png')
        plt.close()

    except Exception as e:
        print(f"Error training {name}: {str(e)}")
    finally:
        gc.collect()
        plt.close('all')

print("\nFinal Model Results Dictionary:")
print(model_results)

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_52/1217787553.py:18: DtypeWarning: Columns (19,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,87,90,91,96,97,98,99,100,101,102,103,104,105,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,127,128,129,131,132,133,134,138) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/work/merged_data.csv')
Sample wnd values: ['220,5,N,0036,5', '200,5,N,0051,5', '220,5,N,0036,5', '200,5,N,0051,5', '200,1,N,0041,1', '210,5,N,0046,5', '210,5,N,0041,5', '220,5,N,0031,5', '220,5,N,0031,5', '220,5,N,0051,5']
Sample dep_hour values: ['2022-01-01 10:00:00', '2022-01-01 21:00:00', '2022-01-01 10:00:00', '2022-01-01 21:00:00'

ModuleNotFoundError: No module named 'xgboost'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f5a99cd0-ffba-48ec-b167-e16ae5f7239b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>